In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import os
import glob
import seaborn as sns

In [ ]:
DATA_DIR = "./data"

input_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/input_2023_w*.csv")))
output_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/output_2023_w*.csv")))

df_in = pd.concat((pd.read_csv(p) for p in tqdm(input_files, desc="loading inputs")), ignore_index=True)
df_out = pd.concat((pd.read_csv(p) for p in tqdm(output_files, desc="loading outputs")), ignore_index=True)

test_in = pd.read_csv(os.path.join(DATA_DIR, "test_input.csv"))
test_template = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sample_submission = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

print("Inputs:", df_in.shape, "Outputs:", df_out.shape, "Test input:", test_in.shape)

In [ ]:
final_frame = df_in.groupby(['game_id','play_id','nfl_id']).tail(1)
player_predict_final = final_frame[final_frame['player_to_predict'] == True]
player_predict_final = player_predict_final[['game_id','play_id','nfl_id','frame_id']]

In [ ]:
df_in = df_in.copy()  # avoid SettingWithCopyWarning

is_final_frame = pd.MultiIndex.from_frame(df_in[['game_id','play_id','nfl_id','frame_id']]).isin(
    pd.MultiIndex.from_frame(player_predict_final)
)

df_in['final_frame_prediction'] = np.where(is_final_frame, 1, 0)

In [ ]:
def feature_engineer(df):
    '''Function taking in the input features / prethrow frames and computing relevant features up to the final prethrow frame'''
    pass


def merge_features(df):
    '''merge snapshots with df_out for training'''
    pass

